# Kickstarter Campaign Success Predictor
Can we use machine learning to predict the success of a Kickstarter campaign before it is released?

## Step 1: Install Libraries and Adjust Settings
In the following code blocks, I imported Python libraries that let me read and clean the data. I also install libraries to visualize the data and convert date columns to datetime.

I imported the Warnings library to ignore the depreciation warnings, and adjusted the display window to display all columns and 100 rows in a dataset.

In [86]:
# Installing initial libraries for data cleanup & visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [87]:
# Ignore warnings / Cleans up output
import warnings 
warnings.filterwarnings('ignore')

In [88]:
pd.set_option('display.max_columns', None) # See all columns
pd.set_option('display.max_rows', 100) # More rows visible

## Step 2: Preview the Data and Adjust Column DataTypes
The following code blocks:
- previews the dataframes
- checks the column datatypes and missing values
- removes the whitespace from the end of the column names
- describes the dataset using statistics like count, mean, std, etc.

In [89]:
# Load and preview the data
# df = pd.read_csv('C:/Users/jlynn/Data_Projects/kickstarter_predictive_modeling/data/raw/ks-projects-201612.csv',encoding='latin-1')
df = pd.read_csv('C:/Users/jlynn/Data_Projects/kickstarter_predictive_modeling/data/raw/ks-projects-201612.csv', encoding='latin-1', quotechar='"', escapechar='\\')
print(f'Dataset shape: {df.shape}')
df.head()

Dataset shape: (323750, 17)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


In [90]:
# Check data types and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323750 entries, 0 to 323749
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              323750 non-null  int64  
 1   name            323746 non-null  object 
 2   category        323745 non-null  object 
 3   main_category   323750 non-null  object 
 4   currency        323750 non-null  object 
 5   deadline        323750 non-null  object 
 6   goal            323750 non-null  object 
 7   launched        323750 non-null  object 
 8   pledged         323750 non-null  object 
 9   state           323750 non-null  object 
 10  backers         323750 non-null  object 
 11  country         323749 non-null  object 
 12  usd pledged     319958 non-null  object 
 13  Unnamed: 13     625 non-null     object 
 14  Unnamed: 14     12 non-null      object 
 15  Unnamed: 15     4 non-null       object 
 16  Unnamed: 16     1 non-null       float64
dtypes: float64

In [91]:
# Find column names
df.columns

Index(['ID ', 'name ', 'category ', 'main_category ', 'currency ', 'deadline ',
       'goal ', 'launched ', 'pledged ', 'state ', 'backers ', 'country ',
       'usd pledged ', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

In [92]:
# Rename columns (removing spaces)
df.columns = df.columns.str.strip()
df.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

In [93]:
# Summary statistics
df.describe()

,ID,Unnamed: 16
count,3.237500e+05,1.0
mean,1.074943e+09,25.0
std,6.193352e+08,NaN
min,5.971000e+03,25.0
25%,5.375802e+08,25.0
50%,1.075818e+09,25.0
75%,1.611070e+09,25.0
max,2.147476e+09,25.0


In [94]:
# Find total number of missing values by column
df.isnull().sum()

ID                    0
name                  4
category              5
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               1
usd pledged        3792
Unnamed: 13      323125
Unnamed: 14      323738
Unnamed: 15      323746
Unnamed: 16      323749
dtype: int64

## Step 3: Clean the DataFrame
After previewing the dataset, I noticed that there were 4 erroneous columns ('Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'). Further investigation revealed that some of the values in the 'name' column had been split across multiple columns due to delimiters, forcing the other columns' values to shift to the right.

These next code blocks clean the data so that the data is in the right columns, and the 'Unnamed' columns can be removed from the dataset.

In [95]:
# Look at rows with missing values
# NaN values in 'name' column are okay
# NaN values in 'category' column are problematic - Data needs to be shifted to the left
df[df['category'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
36671,1218074363,I am Rupert,NaN,Webseries,Film & Video,GBP,2014-05-16 22:25:57,2000,2014-03-27 21:25:57,5,failed,1,GB,8.26779725,NaN,NaN,NaN
41069,124438738,BlanketPals TM,NaN,Interactive Design,Design,USD,2015-03-25 20:51:48,9750,2015-02-23 21:51:48,10890.45,successful,107,US,10890.45,NaN,NaN,NaN
63544,1378236004,{string&&loop} Knitting Craft with Code,NaN,Apparel,Fashion,USD,2016-05-28 00:00:00,1800,2016-04-29 21:04:40,2308,successful,119,US,2308,NaN,NaN,NaN
96753,1574873938,American Pin-up Revisitedd,NaN,Art,Art,USD,2014-04-03 17:52:09,7500,2014-03-04 17:52:09,100,failed,1,US,100.0,NaN,NaN,NaN
269930,676846639,Uplift: The Wearable iPhone Case with Zipline&...,NaN,Design,Design,USD,2011-04-04 20:08:17,50000,2011-03-05 20:08:17,1665,canceled,10,US,1665,NaN,NaN,NaN


In [96]:
# Identify rows with missing values in the 'category' column
shifted_mask = df['category'].isnull()

# Shift data to the left for rows with missing 'category' values
df.loc[shifted_mask, 'category'] = df.loc[shifted_mask, 'main_category'].values

# Verify the shift
df[df['category'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16


In [97]:
# Find updated total number of missing values by column
df.isnull().sum()

ID                    0
name                  4
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               1
usd pledged        3792
Unnamed: 13      323125
Unnamed: 14      323738
Unnamed: 15      323746
Unnamed: 16      323749
dtype: int64

In [98]:
# Look at rows with missing values
df[df['country'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
17912,1105671266,"O Paradiso 2015 Vinyl Record Series ////,Music",Music,USD,2015-04-06 04:00:00,3000,2015-03-11 17:41:24,3136,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# Check how many rows have matching category and main_category
matching = df[df['category'] == df['main_category']]
print(f"Rows where category == main_category: {len(matching)} out of {len(df)}")
print(f"Percentage: {len(matching)/len(df)*100:.1f}%")

Rows where category == main_category: 90078 out of 323750
Percentage: 27.8%


In [100]:
# Find rows where country is null (these need shifting right)
needs_fix = df['country'].isna()

print(f"Rows needing fix: {needs_fix.sum()}")

# Shift each column individually (work backwards)
df.loc[needs_fix, 'Unnamed: 16'] = df.loc[needs_fix, 'Unnamed: 15'].values
df.loc[needs_fix, 'Unnamed: 15'] = df.loc[needs_fix, 'Unnamed: 14'].values
df.loc[needs_fix, 'Unnamed: 14'] = df.loc[needs_fix, 'Unnamed: 13'].values
df.loc[needs_fix, 'Unnamed: 13'] = df.loc[needs_fix, 'usd pledged'].values
df.loc[needs_fix, 'usd pledged'] = df.loc[needs_fix, 'country'].values
df.loc[needs_fix, 'country'] = df.loc[needs_fix, 'backers'].values
df.loc[needs_fix, 'backers'] = df.loc[needs_fix, 'state'].values
df.loc[needs_fix, 'state'] = df.loc[needs_fix, 'pledged'].values
df.loc[needs_fix, 'pledged'] = df.loc[needs_fix, 'launched'].values
df.loc[needs_fix, 'launched'] = df.loc[needs_fix, 'goal'].values
df.loc[needs_fix, 'goal'] = df.loc[needs_fix, 'deadline'].values
df.loc[needs_fix, 'deadline'] = df.loc[needs_fix, 'currency'].values
df.loc[needs_fix, 'currency'] = df.loc[needs_fix, 'main_category'].values
df.loc[needs_fix, 'main_category'] = df.loc[needs_fix, 'category'].values

# Duplicate main_category into category (since it shifted right)
df.loc[needs_fix, 'category'] = df.loc[needs_fix, 'main_category']

# Verify
print(f"\nNull country after fix: {df['country'].isna().sum()}")
df[needs_fix].head()

Rows needing fix: 1

Null country after fix: 0


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
17912,1105671266,"O Paradiso 2015 Vinyl Record Series ////,Music",Music,Music,USD,2015-04-06 04:00:00,3000,2015-03-11 17:41:24,3136,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN


In [101]:
# Find updated total number of missing values by column
df.isnull().sum()

ID                    0
name                  4
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd pledged        3792
Unnamed: 13      323125
Unnamed: 14      323738
Unnamed: 15      323746
Unnamed: 16      323749
dtype: int64

In [102]:
# Look at rows with missing values
df[df['usd pledged'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
150,1000694855,STREETFIGHTERZ WHEELIE MURICA,Film & Video,Film & Video,USD,2014-09-20 06:59:00,6500,2014-08-06 21:28:36,555,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
287,100149523,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25 23:00:00,4500,2015-08-04 12:05:17,4767,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
549,1003023003,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09 21:06:13,3500,2015-03-10 20:06:13,3576,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
561,1003130892,Butter Side Down Debut Album,Music,Music,USD,2015-11-26 10:59:00,6000,2015-11-02 22:09:19,7007.8,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
650,1003629045,Chase Goehring debut EP,Music,Music,USD,2016-03-21 06:00:00,3000,2016-02-23 03:09:49,3660.38,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323391,997971307,"EQUUS International Film Festival®, all-equine...",Film & Video,Film & Video,USD,2015-02-03 04:19:14,7500,2014-12-05 04:19:14,10,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
323456,998319149,Emily 2050 - Short Film,Film & Video,Film & Video,CAD,2014-05-23 00:30:09,3000,2014-04-08 00:30:09,3102,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
323561,9988744,Matthew Stephens Music,Music,Music,USD,2016-02-05 23:00:00,5000,2016-01-06 21:59:23,235,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN
323683,999610349,Lady Vendredi: Afrofuturist concept 12 inch EP,Music,Music,GBP,2015-10-19 22:33:18,2000,2015-09-21 22:33:18,2125,undefined,0,"N,""0",NaN,NaN,NaN,NaN,NaN


In [103]:
# Replace NaN values in 'usd pledged' with 0
df['usd pledged'] = df['usd pledged'].fillna(0)

# Verify
df[df['usd pledged'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16


In [104]:
# Find rows where country is 'N,"0' or starts with 'N,'
mask = df['country'].str.contains('N,', na=False)

print(f"Rows with malformed country: {mask.sum()}")

# Replace country with first 2 characters of currency for those rows
df.loc[mask, 'country'] = df.loc[mask, 'currency'].str[:2]

# Verify the fix
print(f"\nCountries after fix:")
print(df.loc[mask, ['country', 'currency']].head())
# Check if any 'N,"0' values remain
print(f"\nRemaining 'N,\"0' values: {(df['country'] == 'N,\"0').sum()}")

Rows with malformed country: 3790

Countries after fix:
    country currency
150      US      USD
287      AU      AUD
549      US      USD
561      US      USD
650      US      USD

Remaining 'N,"0' values: 0


In [105]:
# Find updated total number of missing values by column
df.isnull().sum()

ID                    0
name                  4
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd pledged           0
Unnamed: 13      323125
Unnamed: 14      323738
Unnamed: 15      323746
Unnamed: 16      323749
dtype: int64

In [106]:
# Look at the Unnamed columns for any remaining values
# Unnamed: 13 is not empty because 'name' and 'category' should be combined into one column, and 'main_category' shifted right one time
# Unnamed: 14 is not empyt because 'name', 'category', and 'main_category' should be combined into one column, and 'currency' shifted right two times
# Unnamed: 15 is not empty because 'name', 'category', 'main_category', and 'currency' should be combined into one column, and 'deadline' shifted right three times
# Unnamed: 16 is not empty because 'name', 'category', 'main_category', 'currency', and 'deadline' should be combined into one column, and 'goal' shifted right four times
df[df['Unnamed: 16'].notna()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
269970,677103185,SixSixSeven,Angels,Demons,Religion,Esoteric,Graphic Novels,Comics,USD,2015-10-10 01:00:00,750,2015-09-10 18:15:45,25,failed,1,US,25.0


In [107]:
# Fix rows where Unnamed: 16 has data (most broken - shifted 4 columns)
mask_16 = df['Unnamed: 16'].notna()
print(f"Rows with Unnamed: 16 data: {mask_16.sum()}")

# For these rows:
# 1. Combine name through deadline into the name column
df.loc[mask_16, 'name'] = (df.loc[mask_16, 'name'].astype(str) + ', ' + 
                             df.loc[mask_16, 'category'].astype(str) + ', ' + 
                             df.loc[mask_16, 'main_category'].astype(str) + ', ' + 
                             df.loc[mask_16, 'currency'].astype(str) + ', ' + 
                             df.loc[mask_16, 'deadline'].astype(str))
# 2. Shift everything from goal onwards back to category (4 positions left)
df.loc[mask_16, 'category'] = df.loc[mask_16, 'goal'].values
df.loc[mask_16, 'main_category'] = df.loc[mask_16, 'launched'].values
df.loc[mask_16, 'currency'] = df.loc[mask_16, 'pledged'].values
df.loc[mask_16, 'deadline'] = df.loc[mask_16, 'state'].values
df.loc[mask_16, 'goal'] = df.loc[mask_16, 'backers'].values
df.loc[mask_16, 'launched'] = df.loc[mask_16, 'country'].values
df.loc[mask_16, 'pledged'] = df.loc[mask_16, 'usd pledged'].values
df.loc[mask_16, 'state'] = df.loc[mask_16, 'Unnamed: 13'].values
df.loc[mask_16, 'backers'] = df.loc[mask_16, 'Unnamed: 14'].values
df.loc[mask_16, 'country'] = df.loc[mask_16, 'Unnamed: 15'].values
df.loc[mask_16, 'usd pledged'] = df.loc[mask_16, 'Unnamed: 16'].values

# 3. Clear the Unnamed columns for these rows
df.loc[mask_16, ['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']] = None

# Verify
print(f"\nRemaining Unnamed: 16 values: {df['Unnamed: 16'].notna().sum()}")
df[mask_16].head()

Rows with Unnamed: 16 data: 1

Remaining Unnamed: 16 values: 0


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
269970,677103185,"SixSixSeven , Angels , Demons , Religion , ...",Graphic Novels,Comics,USD,2015-10-10 01:00:00,750,2015-09-10 18:15:45,25,failed,1,US,25.0,None,None,None,None


In [108]:
# Fix rows where Unnamed: 15 has data (shifted 3 columns)
mask_15 = df['Unnamed: 15'].notna()
print(f"Rows with Unnamed: 15 data: {mask_15.sum()}")

# For these rows:
# 1. Combine name through currency into the name column
df.loc[mask_15, 'name'] = (df.loc[mask_15, 'name'].astype(str) + ', ' + 
                             df.loc[mask_15, 'category'].astype(str) + ', ' + 
                             df.loc[mask_15, 'main_category'].astype(str) + ', ' + 
                             df.loc[mask_15, 'currency'].astype(str))

# 2. Shift everything from deadline onwards back to category (3 positions left)
df.loc[mask_15, 'category'] = df.loc[mask_15, 'deadline'].values
df.loc[mask_15, 'main_category'] = df.loc[mask_15, 'goal'].values
df.loc[mask_15, 'currency'] = df.loc[mask_15, 'launched'].values
df.loc[mask_15, 'deadline'] = df.loc[mask_15, 'pledged'].values
df.loc[mask_15, 'goal'] = df.loc[mask_15, 'state'].values
df.loc[mask_15, 'launched'] = df.loc[mask_15, 'backers'].values
df.loc[mask_15, 'pledged'] = df.loc[mask_15, 'country'].values
df.loc[mask_15, 'state'] = df.loc[mask_15, 'usd pledged'].values
df.loc[mask_15, 'backers'] = df.loc[mask_15, 'Unnamed: 13'].values
df.loc[mask_15, 'country'] = df.loc[mask_15, 'Unnamed: 14'].values
df.loc[mask_15, 'usd pledged'] = df.loc[mask_15, 'Unnamed: 15'].values

# 3. Clear the Unnamed columns for these rows
df.loc[mask_15, ['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15']] = None

# Verify
print(f"\nRemaining Unnamed: 15 values: {df['Unnamed: 15'].notna().sum()}")
df[mask_15].head()

Rows with Unnamed: 15 data: 3



Remaining Unnamed: 15 values: 0


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
104120,1618382802,"Druid Hill Park Passport: Discover, Enjoy, L...",Publishing,Publishing,USD,2012-07-28 01:30:00,9500,2012-06-06 23:54:14,9854,successful,208,US,9854.0,None,None,None,NaN
145502,1864561559,"T-shirt, dress shirts, pants, clothing company",Apparel,Fashion,USD,2015-08-02 18:23:23,20000,2015-07-03 18:23:23,0,failed,0.0,US,0.0,None,None,None,NaN
264243,642928449,"FRANCIS - The Pope, Buenos Aires, soccer, t...",Film & Video,Film & Video,USD,2014-03-02 23:34:00,75000,2014-02-05 02:06:20,0,failed,0,US,0,None,None,None,NaN


In [109]:
# Fix rows where Unnamed: 14 has data (shifted 2 columns)
mask_14 = df['Unnamed: 14'].notna()
print(f"Rows with Unnamed: 14 data: {mask_14.sum()}")

# For these rows:
# 1. Combine name through main_category into the name column
df.loc[mask_14, 'name'] = (df.loc[mask_14, 'name'].astype(str) + ', ' + 
                             df.loc[mask_14, 'category'].astype(str) + ', ' + 
                             df.loc[mask_14, 'main_category'].astype(str))

# 2. Shift everything from currency onwards back to category (2 positions left)
df.loc[mask_14, 'category'] = df.loc[mask_14, 'currency'].values
df.loc[mask_14, 'main_category'] = df.loc[mask_14, 'deadline'].values
df.loc[mask_14, 'currency'] = df.loc[mask_14, 'goal'].values
df.loc[mask_14, 'deadline'] = df.loc[mask_14, 'launched'].values
df.loc[mask_14, 'goal'] = df.loc[mask_14, 'pledged'].values
df.loc[mask_14, 'launched'] = df.loc[mask_14, 'state'].values
df.loc[mask_14, 'pledged'] = df.loc[mask_14, 'backers'].values
df.loc[mask_14, 'state'] = df.loc[mask_14, 'country'].values
df.loc[mask_14, 'backers'] = df.loc[mask_14, 'usd pledged'].values
df.loc[mask_14, 'country'] = df.loc[mask_14, 'Unnamed: 13'].values
df.loc[mask_14, 'usd pledged'] = df.loc[mask_14, 'Unnamed: 14'].values

# 3. Clear the Unnamed columns for these rows
df.loc[mask_14, ['Unnamed: 13', 'Unnamed: 14']] = None

# Verify
print(f"\nRemaining Unnamed: 14 values: {df['Unnamed: 14'].notna().sum()}")
df[mask_14].head()

Rows with Unnamed: 14 data: 8

Remaining Unnamed: 14 values: 0


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
13795,1081139420,"The Rolling Stones, BEGGARS BANQUET, 50 Year...",Rock,Music,USD,2011-08-12 01:17:48,4625,2011-06-04 01:17:48,20,failed,2,US,20.0,None,None,NaN,NaN
16482,1097082409,"Pixel art, personalized, Retro Gaming art.",Crafts,Crafts,USD,2016-07-28 21:36:59,500,2016-06-28 21:36:59,0,failed,0,US,0.0,None,None,NaN,NaN
61447,1365419849,"VIA, ME (V.isual I.nteractive A.rchive, M.er...",Software,Technology,USD,2016-03-19 20:40:23,10000,2016-02-18 21:40:23,26,failed,2,US,26.0,None,None,NaN,NaN
112934,1670090635,"EXCALIBRE:Sword of Truth, Path of Light, Kin...",Nonfiction,Publishing,USD,2016-04-01 08:00:00,24000,2016-02-22 21:57:30,1,failed,1,US,1,None,None,NaN,NaN
225692,410977509,"Handmade Artisan Soaps: Natural, Gentle, Goo...",Crafts,Crafts,USD,2015-10-08 22:08:39,200,2015-09-08 22:08:39,435,successful,10,US,435.0,None,None,NaN,NaN


In [110]:
# Fix rows where Unnamed: 13 has data (shifted 1 column)
mask_13 = df['Unnamed: 13'].notna()
print(f"Rows with Unnamed: 13 data: {mask_13.sum()}")

# For these rows:
# 1. Combine name and category into the name column
df.loc[mask_13, 'name'] = (df.loc[mask_13, 'name'].astype(str) + ', ' + 
                             df.loc[mask_13, 'category'].astype(str))

# 2. Shift everything from main_category onwards back to category (1 position left)
df.loc[mask_13, 'category'] = df.loc[mask_13, 'main_category'].values
df.loc[mask_13, 'main_category'] = df.loc[mask_13, 'currency'].values
df.loc[mask_13, 'currency'] = df.loc[mask_13, 'deadline'].values
df.loc[mask_13, 'deadline'] = df.loc[mask_13, 'goal'].values
df.loc[mask_13, 'goal'] = df.loc[mask_13, 'launched'].values
df.loc[mask_13, 'launched'] = df.loc[mask_13, 'pledged'].values
df.loc[mask_13, 'pledged'] = df.loc[mask_13, 'state'].values
df.loc[mask_13, 'state'] = df.loc[mask_13, 'backers'].values
df.loc[mask_13, 'backers'] = df.loc[mask_13, 'country'].values
df.loc[mask_13, 'country'] = df.loc[mask_13, 'usd pledged'].values
df.loc[mask_13, 'usd pledged'] = df.loc[mask_13, 'Unnamed: 13'].values

# 3. Clear Unnamed: 13 for these rows
df.loc[mask_13, 'Unnamed: 13'] = None

# Verify ALL Unnamed columns are now clean
print(f"\nRemaining Unnamed: 13 values: {df['Unnamed: 13'].notna().sum()}")
print(f"Remaining Unnamed: 14 values: {df['Unnamed: 14'].notna().sum()}")
print(f"Remaining Unnamed: 15 values: {df['Unnamed: 15'].notna().sum()}")
print(f"Remaining Unnamed: 16 values: {df['Unnamed: 16'].notna().sum()}")

# Final shape check
print(f"\nDataset shape: {df.shape}")
df[mask_13].head()

Rows with Unnamed: 13 data: 613

Remaining Unnamed: 13 values: 0
Remaining Unnamed: 14 values: 0
Remaining Unnamed: 15 values: 0
Remaining Unnamed: 16 values: 0

Dataset shape: (323750, 17)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
1454,1008705746,"Zephyra´s new full length, 'As The World Coll...",Metal,Music,SEK,2016-02-02 00:56:46,15000,2016-01-03 00:56:46,4262,failed,14,SE,504.94765278,None,NaN,NaN,NaN
1563,1009317190,"French Cuisine, A Traditional Experience",Cookbooks,Food,USD,2014-09-08 00:46:23,13730,2014-08-09 03:16:02,3984,failed,46,US,3984,None,NaN,NaN,NaN
1794,1010871699,"The Beginners Guide to being Unsuicidal, the ...",Theater,Theater,USD,2011-12-31 23:25:46,5000,2011-11-21 23:25:46,525,failed,10,US,525,None,NaN,NaN,NaN
1931,1011687764,"Best OnLine Classifieds, Ever / No More Spam",Web,Technology,USD,2014-09-20 19:56:10,6300,2014-08-21 19:56:10,0,failed,0,US,0,None,NaN,NaN,NaN
2420,101453314,"Social Media Ruined My Life, A Short Film fro...",Shorts,Film & Video,USD,2013-03-14 20:11:57,3000,2013-02-25 21:11:57,3035,successful,42,US,3035,None,NaN,NaN,NaN


In [111]:
# Drop Unnamed columns
df = df.drop(columns=['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
print(f"\nDataset shape after dropping Unnamed columns: {df.shape}")


Dataset shape after dropping Unnamed columns: (323750, 13)


In [112]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375


## Step 4: Checking 'category' and 'main_category' values to catch 'name' outliers
After removing the 'Unnamed' columns from the dataset, I wanted to make sure that there were no outliers in the 'category' and 'main_category' columns where the 'name' category' had split into, but hadn't shifted the columns' data.

These next code blocks check for those outliers and cleans the columns.

In [113]:
# Get value counts to see what the real categories are
category_counts = df['category'].value_counts()
print(f"Total unique categories: {len(category_counts)}")
print("\nTop 20 categories by count:")
print(category_counts.head(20))

Total unique categories: 165

Top 20 categories by count:
category
Product Design      17506
Documentary         14919
Music               13930
Shorts              11694
Tabletop Games      10722
Food                10564
Video Games         10071
Film & Video         9222
Fiction              8266
Fashion              7927
Nonfiction           7432
Art                  6911
Theater              6843
Rock                 6349
Technology           5774
Children's Books     5669
Photography          5526
Apparel              5490
Indie Rock           5378
Webseries            5354
Name: count, dtype: int64


In [114]:
# Show the categories with the lowest counts (likely junk)
print("\nBottom 10 categories by count:")
print(category_counts.tail(10))

# Also check: how many categories appear only 1-5 times?
low_count_cats = category_counts[category_counts <= 5]
print(f"\nCategories appearing 5 times or less: {len(low_count_cats)}")
print(low_count_cats)


Bottom 10 categories by count:
category
Letterpress                            36
Chiptune                               33
Taxidermy                              11
 Marc & Mary sing together at last!     1
 a day in the life of a champion        1
 The North (2015)                       1
 Songs by & about her + Stories         1
 The Sobering Truth                     1
 Right and Wrong                        1
 Recording Soundtracks                  1
Name: count, dtype: int64

Categories appearing 5 times or less: 7
category
Marc & Mary sing together at last!    1
a day in the life of a champion       1
The North (2015)                      1
Songs by & about her + Stories        1
The Sobering Truth                    1
Right and Wrong                       1
Recording Soundtracks                 1
Name: count, dtype: int64


In [115]:
# Create a whitelist of legitimate categories (appeared more than 10 times)
valid_categories = category_counts[category_counts > 10].index.tolist()
print(f"Valid categories (>= 10 occurrences): {len(valid_categories)}")

# Strip whitespace from valid categories list
valid_categories = [cat.strip() for cat in valid_categories]

# Find rows where category is NOT in the valid list
invalid_cat_mask = ~df['category'].str.strip().isin(valid_categories)
print(f"\nRows with invalid categories: {invalid_cat_mask.sum()}")

# Look at some examples
print("\nSample of rows with invalid categories:")
df[invalid_cat_mask][['name', 'category', 'main_category']].head(20)

Valid categories (>= 10 occurrences): 158

Rows with invalid categories: 7

Sample of rows with invalid categories:


,name,category,main_category
64484,CD: Heartsong Harmonics,Marc & Mary sing together at last!,Music
85619,Everyday Beautiful,a day in the life of a champion,Film & Video
98614,Hana Zara's new album,The North (2015),Music
123506,Celebrating Joni Mitchell,Songs by & about her + Stories,Music
159427,Legal Highs,The Sobering Truth,Film & Video
163067,Puppy Training Steps,Right and Wrong,Film & Video
177636,Adventure to Peru's Sacred Valley,Recording Soundtracks,Music


In [116]:
# Combine name and category
df.loc[invalid_cat_mask, 'name'] = df.loc[invalid_cat_mask, 'name'].astype(str) + ': ' + df.loc[invalid_cat_mask, 'category'].astype(str)

# Shift left - move main_category through usd pledged one column to the left
df.loc[invalid_cat_mask, 'category'] = df.loc[invalid_cat_mask, 'main_category'].values
df.loc[invalid_cat_mask, 'main_category'] = df.loc[invalid_cat_mask, 'currency'].values
df.loc[invalid_cat_mask, 'currency'] = df.loc[invalid_cat_mask, 'deadline'].values
df.loc[invalid_cat_mask, 'deadline'] = df.loc[invalid_cat_mask, 'goal'].values
df.loc[invalid_cat_mask, 'goal'] = df.loc[invalid_cat_mask, 'launched'].values
df.loc[invalid_cat_mask, 'launched'] = df.loc[invalid_cat_mask, 'pledged'].values
df.loc[invalid_cat_mask, 'pledged'] = df.loc[invalid_cat_mask, 'state'].values
df.loc[invalid_cat_mask, 'state'] = df.loc[invalid_cat_mask, 'backers'].values
df.loc[invalid_cat_mask, 'backers'] = df.loc[invalid_cat_mask, 'country'].values
df.loc[invalid_cat_mask, 'country'] = df.loc[invalid_cat_mask, 'usd pledged'].values
df.loc[invalid_cat_mask, 'usd pledged'] = None  # This column becomes empty after shift

# Verify
print("Fixed rows:")
df.loc[invalid_cat_mask, ['name', 'category', 'main_category', 'currency']].head(10)

# Check that all categories are now valid
remaining_invalid = ~df['category'].str.strip().isin(valid_categories)
print(f"\nRemaining invalid categories: {remaining_invalid.sum()}")

Fixed rows:

Remaining invalid categories: 0


In [117]:
# Check main_category value counts
main_cat_counts = df['main_category'].value_counts()
print(f"Total unique main categories: {len(main_cat_counts)}")
print("\nAll main categories:")
print(main_cat_counts)

Total unique main categories: 17

All main categories:
main_category
Film & Video    57746
Music           46791
Publishing      34346
Games           28041
Technology      26161
Art             24023
Design          23909
Food            21242
Fashion         18416
Theater          9975
Photography      9689
Comics           8757
Crafts           7192
Journalism       4083
Dance            3377
USD                 1
GBP                 1
Name: count, dtype: int64


In [118]:
# Find rows where main_category is a currency code
currency_mask = df['main_category'].isin(['USD', 'GBP'])
print(f"Rows with currency as main_category: {currency_mask.sum()}")

# Look at these rows
print("\nProblem rows:")
df[currency_mask][['name', 'category', 'main_category', 'currency', 'deadline']].head()

Rows with currency as main_category: 2

Problem rows:


,name,category,main_category,currency,deadline
177079,"///\ Young Hunter Album ///\,Indie Rock",Music,USD,2015-05-12 00:03:53,4444
318586,"'Mapping Budapest' /?Bu?d?p?st,Concep...",Art,GBP,2015-03-08 21:28:02,500


In [119]:
# 1. Split name at the last comma to separate actual name from category
split_data = df.loc[currency_mask, 'name'].str.rsplit(',', n=1, expand=True)
actual_name = split_data[0]
actual_category = split_data[1].str.strip()

# 2. Shift everything from currency onwards one column to the RIGHT
df.loc[currency_mask, 'usd pledged'] = df.loc[currency_mask, 'country'].values
df.loc[currency_mask, 'country'] = df.loc[currency_mask, 'backers'].values
df.loc[currency_mask, 'backers'] = df.loc[currency_mask, 'state'].values
df.loc[currency_mask, 'state'] = df.loc[currency_mask, 'pledged'].values
df.loc[currency_mask, 'pledged'] = df.loc[currency_mask, 'launched'].values
df.loc[currency_mask, 'launched'] = df.loc[currency_mask, 'goal'].values
df.loc[currency_mask, 'goal'] = df.loc[currency_mask, 'deadline'].values
df.loc[currency_mask, 'deadline'] = df.loc[currency_mask, 'currency'].values
df.loc[currency_mask, 'currency'] = df.loc[currency_mask, 'main_category'].values
df.loc[currency_mask, 'main_category'] = df.loc[currency_mask, 'category'].values

# 3. Assign the corrected name and category
df.loc[currency_mask, 'name'] = actual_name
df.loc[currency_mask, 'category'] = actual_category

# Verify
print("Fixed rows:")
df[currency_mask][['name', 'category', 'main_category', 'currency', 'deadline']].head()

# Check main_category again
main_cat_check = df['main_category'].value_counts()
print(f"\nMain categories after fix: {len(main_cat_check)}")
print(main_cat_check)

Fixed rows:

Main categories after fix: 15
main_category
Film & Video    57746
Music           46792
Publishing      34346
Games           28041
Technology      26161
Art             24024
Design          23909
Food            21242
Fashion         18416
Theater          9975
Photography      9689
Comics           8757
Crafts           7192
Journalism       4083
Dance            3377
Name: count, dtype: int64


## Step 5: Remove Whitespaces and Convert 'N,"0' Value in 'country' column

In these next few code blocks, I stripped the whitespaces from all the text columns and converted the erroneous 'N,"0' value in the 'country' column to a 2-character value based off its' 'currency' column.

In [ ]:
# Strip whitespace from all text columns
text_columns = ['name', 'category', 'main_category', 'currency', 'state', 'country']
for col in text_columns:
    df[col] = df[col].str.strip()

# Verify the final shape and check for any remaining issues
print(f"Final dataset shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
print(f"\nMissing values per column:")
print(df.isnull().sum())
print(f"\nData types:")
print(df.dtypes)

df.head(10)

Final dataset shape: (323750, 13)

Column names: ['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged']

Missing values per column:
ID               0
name             4
category         0
main_category    0
currency         0
deadline         0
goal             0
launched         0
pledged          0
state            0
backers          0
country          0
usd pledged      7
dtype: int64

Data types:
ID                int64
name             object
category         object
main_category    object
currency         object
deadline         object
goal             object
launched         object
pledged          object
state            object
backers          object
country          object
usd pledged      object
dtype: object


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375
5,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 18:30:44,1000,2014-12-01 18:30:44,1205,successful,16,US,1205
6,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17 19:05:12,25000,2016-02-01 20:05:12,453,failed,40,US,453
7,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29 18:14:43,125000,2014-04-24 18:14:43,8233,canceled,58,US,8233
8,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10 21:55:48,65000,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57
9,100004721,Of Jesus and Madmen,Nonfiction,Publishing,CAD,2013-10-09 18:19:37,2500,2013-09-09 18:19:37,0,failed,0,CA,0


In [122]:
# Check for the N,"0 issue in country
weird_country = df['country'].str.contains('N,', na=False)
print(f"Rows with 'N,\"0' in country: {weird_country.sum()}")

# Fix them if they exist
if weird_country.sum() > 0:
    df.loc[weird_country, 'country'] = df.loc[weird_country, 'currency'].str[:2]
    print("Fixed country values")


Rows with 'N,"0' in country: 7
Fixed country values

=== COLUMN VALUE CHECKS ===

Unique currencies (13):
currency
USD    260817
GBP     28033
CAD     12214
EUR     11567
AUD      6362
SEK      1302
NZD      1165
DKK       841
NOK       540
CHF       478
MXN       214
SGD       120
HKD        97
Name: count, dtype: int64


Unique states (6):
state
failed        168608
successful    113248
canceled       32414
live            4439
undefined       3562
suspended       1479
Name: count, dtype: int64


Unique countries (22):
country
MX    214
EU    186
SG    120
HK     97
LU     40
Name: count, dtype: int64


=== CHECKING FOR ANOMALIES ===

Goal column data type: float64
Pledged column data type: float64
Backers column data type: Int64


## Step 5: Convert DataTypes, Column Validation, and Export Cleaned Dataset
Final step was to convert the numeric and date columns to their appropriate datatype, complete a final validation on the columns' data for any wierd values, and then export the cleaned dataset as a CSV file.

In [121]:
print('Converting data types...')

# Numeric columns
df['goal'] = pd.to_numeric(df['goal'], errors='coerce')
df['pledged'] = pd.to_numeric(df['pledged'], errors='coerce')
df['usd pledged'] = pd.to_numeric(df['usd pledged'], errors='coerce')
df['backers'] = pd.to_numeric(df['backers'], errors='coerce').astype('Int64')

# Date columns
df['deadline'] = pd.to_datetime(df['deadline'], errors='coerce')
df['launched'] = pd.to_datetime(df['launched'], errors='coerce')

print('\nData types after conversion:')
print(df.dtypes)

Converting data types...

Data types after conversion:
ID                        int64
name                     object
category                 object
main_category            object
currency                 object
deadline         datetime64[ns]
goal                    float64
launched         datetime64[ns]
pledged                 float64
state                    object
backers                   Int64
country                  object
usd pledged             float64
dtype: object


In [ ]:

# Now let's check ALL columns for weird values
print("\n=== COLUMN VALUE CHECKS ===\n")

# Currency - should be 3-letter codes
print(f"Unique currencies ({df['currency'].nunique()}):")
print(df['currency'].value_counts())

print(f"\n\nUnique states ({df['state'].nunique()}):")
print(df['state'].value_counts())

print(f"\n\nUnique countries ({df['country'].nunique()}):")
print(df['country'].value_counts().tail(5))

# Check for any remaining weird characters or patterns
print("\n\n=== CHECKING FOR ANOMALIES ===")
# Check if any numeric columns have text
print(f"\nGoal column data type: {df['goal'].dtype}")
print(f"Pledged column data type: {df['pledged'].dtype}")
print(f"Backers column data type: {df['backers'].dtype}")

In [123]:
df.to_csv('C:/Users/jlynn/Data_Projects/kickstarter_predictive_modeling/data/cleaned/ks-projects-201612-cleaned.csv', index=False)